In [9]:
import requests
import datetime
import json

In [31]:
SUMMARY_INDEX = "gracc.osg.summary"
ENDPOINT = "https://gracc.opensciencegrid.org:443/q"
HEADERS = {
    'Content-Type': 'application/json',
    'authority': 'gracc.opensciencegrid.org',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'en-US,en;q=0.9'
}
SESSION = requests.Session()
SESSION.headers.update(HEADERS)

hasnt_facility_query = {
    "size": 0,
    "query": {
        "bool" : {
            "filter" : [
                { "term" : { "ResourceType": "Payload" }},
                { "range": {
                    "EndTime": {
                        "lte": int(datetime.datetime.now().timestamp()*1000),
                        "gte": int((datetime.datetime.now() - datetime.timedelta(days=365)).timestamp()*1000)
                    }
                }}
            ],
        },
    },
    "aggs": {
        "facility": {
            "terms": {
                "field": "ProjectName",
                "size": 99999999
            },
            "aggs": {
                "projectCpu": {
                    "sum": {
                        "field": "CoreHours"
                    }
                },
                "projectFieldsOfScience" : {
                    "terms": {
                        "field": "ProjectName",
                        "size" : 500
                    }
                },
                "projectGpu": {
                    "sum": {
                        "field": "GPUHours"
                    }
                },
                "gpu_bucket_filter": {
                  "bucket_selector": {
                    "buckets_path": {
                      "totalCPU": "projectCpu",
                      "totalGPU": "projectGpu"
                    },
                    "script": "params.totalGPU > 0 || params.totalCPU > 0"
                  }
                }
            }
        }
    }
}

response = SESSION.post(f"{ENDPOINT}/{SUMMARY_INDEX}/_search", data=json.dumps(hasnt_facility_query))
response_json = response.json()
response_json

{'took': 107,
 'timed_out': False,
 '_shards': {'total': 19, 'successful': 19, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'facility': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'Fermi National Accelerator Laboratory',
     'doc_count': 74568,
     'projectFieldsOfScience': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 0,
      'buckets': [{'key': 'nova', 'doc_count': 11796},
       {'key': 'cms', 'doc_count': 9904},
       {'key': 'dune', 'doc_count': 8721},
       {'key': 'microboone', 'doc_count': 8054},
       {'key': 'gm2', 'doc_count': 6996},
       {'key': 'icarus', 'doc_count': 4828},
       {'key': 'minerva', 'doc_count': 3395},
       {'key': 'sbnd', 'doc_count': 2471},
       {'key': 'mu2e', 'doc_count': 2314},
       {'key': 'fermilab', 'doc_count': 1406},
       {'key': 'cms.org.cern', 'doc_count': 1321},
       {'key':

In [37]:
facility = {x['key']: x for x in response_json['aggregations']['facility']['buckets']}

KeyboardInterrupt: 

In [31]:
site_names_no_facility = set(x['key'] for x in hasnt_facility_response_json['aggregations']['sites']['buckets'])
site_names_no_facility

cpuSum = sum([x['projectCpu']['value'] for x in hasnt_facility_response_json['aggregations']['sites']['buckets']])
gpuSum = sum([x['projectGpu']['value'] for x in hasnt_facility_response_json['aggregations']['sites']['buckets']])

print(f"GPU sum: {gpuSum}", f"\nCPU Sum: {cpuSum}")

GPU sum: 1685180.592217311 
CPU Sum: 678466716.0294442


In [26]:
has_facility_query = {
    "size": 0,
    "query": {
        "bool" : {
            "filter" : [
                { "term" : { "ResourceType": "Payload" }},
                { "range": {
                    "EndTime": {
                        "lte": int(datetime.datetime.now().timestamp()*1000),
                        "gte": int((datetime.datetime.now() - datetime.timedelta(days=365)).timestamp()*1000)
                    }
                }},
                {
                   "bool": {
                      "must": [{ "exists": { "field": "OIM_Facility" } }]
                   }
                },
            ],
        },
    },
    "aggs": {
        "sites": {
            "terms": {
                "field": "OIM_Site",
                "size": 99999999
            },
            "aggs": {
                "projectCpu": {
                    "sum": {
                        "field": "CoreHours"
                    }
                },
                "projectGpu": {
                    "sum": {
                        "field": "GPUHours"
                    }
                },
                "gpu_bucket_filter": {
                  "bucket_selector": {
                    "buckets_path": {
                      "totalCPU": "projectCpu",
                      "totalGPU": "projectGpu"
                    },
                    "script": "params.totalGPU > 0 || params.totalCPU > 0"
                  }
                },
            }
        }
    }
}

has_facility_response_json = requests.get(f"{ENDPOINT}/{SUMMARY_INDEX}/_search", data=json.dumps(has_facility_query), headers=HEADERS).json()

has_facility_response_json

{'took': 63,
 'timed_out': False,
 '_shards': {'total': 19, 'successful': 19, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'sites': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'FermiGrid',
     'doc_count': 58356,
     'projectGpu': {'value': 0.0},
     'projectCpu': {'value': 144295639.54972222}},
    {'key': 'SU ITS',
     'doc_count': 35964,
     'projectGpu': {'value': 3078.7400272339582},
     'projectCpu': {'value': 153445955.49416667}},
    {'key': 'UColorado_HEP',
     'doc_count': 22579,
     'projectGpu': {'value': 0.0},
     'projectCpu': {'value': 9331546.063888889}},
    {'key': 'GLOW',
     'doc_count': 22183,
     'projectGpu': {'value': 7011.5530758058885},
     'projectCpu': {'value': 47349780.97055556}},
    {'key': 'CHTC',
     'doc_count': 19619,
     'projectGpu': {'value': 345114.5431418865},
     'projectCpu': {'value': 84474399.

In [27]:
site_names_has_facility = set(x['key'] for x in has_facility_response_json['aggregations']['sites']['buckets'])
site_names_has_facility

{'AGLT2',
 'AMNH',
 'ASGC',
 'ASU Research Computing',
 'AUGIE UC',
 'Arkansas State University',
 'BNL ATLAS Tier1',
 'BNL-SDCC',
 'Baylor University',
 'CHTC',
 'CU - Research Computing',
 'Caltech CMS Tier2',
 'Center for Advanced Research Computing',
 'Clarkson - ACRES',
 'Clemson-Palmetto',
 'ComputeCanada - Cedar',
 'Creighton University',
 'Cybera - Edmonton',
 'Dakota State University',
 'Emporia State University',
 'FANDM-ITS',
 'FNAL HPC',
 'FNAL USCMS Tier1',
 'FSU_HNPGRID',
 'FZU',
 'FermiGrid',
 'GLOW',
 'GP-ARGO',
 'GSU - ACIDS',
 'Georgia Tech',
 'ICTS Alice Cluster',
 'IIT - Illinois Institute of Technology',
 'INFN-T1',
 'IRISHEP-SSL-UCHICAGO',
 'Information Sciences Institute',
 'JINR_Cloud',
 'JLAB',
 'Kansas State University',
 'LSU - Deep Bayou',
 'LSU QB2',
 'LSU SuperMIC',
 'Lafayette College',
 'Lamar Cluster',
 'Lancium',
 'Langston University',
 'Lehigh - Hawk',
 'Louisiana State University Health Sciences Center',
 'MIT CMS',
 'MWT2 ATLAS UC',
 'Missouri Univ

In [28]:
site_names_has_facility.difference(site_names_no_facility)

{'AGLT2',
 'AMNH',
 'ASGC',
 'ASU Research Computing',
 'AUGIE UC',
 'Arkansas State University',
 'BNL ATLAS Tier1',
 'BNL-SDCC',
 'Baylor University',
 'CHTC',
 'CU - Research Computing',
 'Caltech CMS Tier2',
 'Center for Advanced Research Computing',
 'Clarkson - ACRES',
 'Clemson-Palmetto',
 'ComputeCanada - Cedar',
 'Creighton University',
 'Cybera - Edmonton',
 'Dakota State University',
 'Emporia State University',
 'FANDM-ITS',
 'FNAL HPC',
 'FNAL USCMS Tier1',
 'FSU_HNPGRID',
 'FZU',
 'FermiGrid',
 'GLOW',
 'GP-ARGO',
 'GSU - ACIDS',
 'Georgia Tech',
 'ICTS Alice Cluster',
 'IIT - Illinois Institute of Technology',
 'INFN-T1',
 'IRISHEP-SSL-UCHICAGO',
 'Information Sciences Institute',
 'JINR_Cloud',
 'JLAB',
 'Kansas State University',
 'LSU - Deep Bayou',
 'LSU QB2',
 'LSU SuperMIC',
 'Lafayette College',
 'Lamar Cluster',
 'Lancium',
 'Langston University',
 'Lehigh - Hawk',
 'Louisiana State University Health Sciences Center',
 'MIT CMS',
 'MWT2 ATLAS UC',
 'Missouri Univ